<img src="https://www.unir.net/wp-content/uploads/2019/11/Unir_2021_logo.svg" width="240" height="240" align="right"/>

<center><h1>TRABAJO FIN DE MÁSTER</header1></center>

## Importación de librerias necesarias

In [12]:
from pandas import read_csv
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LassoCV
import statsmodels.api as sm
import scipy.stats as stats

Las variables que componen el índice de vulnerabilidad se seleccionan a partir de los resultados obtenidos en las fases explicativa y predictiva. De este modo, el índice integra factores estructurales asociados a la población municipal, identificados en la fase 1, junto con la dinámica demográfica futura proyectada en la fase 2. Además, se tienen en cuenta los factores principales vistos también en el marco teórico del trabajo.

El índice de vulnerabilidad se estructura en cinco dimensiones que recogen los principales factores demográficos, económicos, territoriales y la dinámica futura de la población, permitiendo una evaluación integral del riesgo de despoblación

1. Dimensión DEMOGRÁFICA

Refleja la estructura poblacional y capacidad de renovación: PROP_ENVEJECIMIENTO

2. Dimensión ECONÓMICA–LABORAL

Capacidad de generar empleo y actividad: EMPRESAS

3. Dimensión SERVICIOS

Acceso a servicios básicos y atractores: SERVICIOS_SANIDAD, ESCUELAS

4. Dimensión ACCESIBILIDAD TERRITORIAL

Aislamiento geográfico: DISTANCIA_CAPITAL

5. Dimensión DINÁMICA FUTURA

Riesgo proyectado de despoblación (núcleo del índice): VAR_2024_2030_NEUTRO_PCT

ntre las variables económicas, se prioriza el número de empresas como indicador de la capacidad productiva estructural del municipio. La tasa de demanda de empleo, de carácter más coyuntural y reactivo, se excluye del índice al no reflejar de forma directa la capacidad de fijación poblacional a medio y largo plazo

In [27]:
df_proyecciones = pd.read_csv("proyeccion_poblacion_2025_2031.csv")

df_proyecciones.head()

,COD_MUNICIPIO,MUNICIPIO,FECHA,POBLACION_PROYECTADA,ESCENARIO
0,42001,42001 ABEJAR,2025,298,NEUTRO
1,42001,42001 ABEJAR,2025,295,PESIMISTA
2,42001,42001 ABEJAR,2026,297,NEUTRO
3,42001,42001 ABEJAR,2026,291,PESIMISTA
4,42001,42001 ABEJAR,2027,295,NEUTRO


Se selecciona el escenario neutro como referencia para la construcción del índice de vulnerabilidad.

In [29]:
df_neutro = df_proyecciones[df_proyecciones["ESCENARIO"] == "NEUTRO"].copy()
#Se selecciona el año 2030 como horizonte temporal para evaluar el riesgo demográfico a medio plazo.
df_2030 = df_neutro[df_neutro["FECHA"] == 2030].copy()

In [30]:
df_modelo = pd.read_csv("df_modelo_POST_EDA.csv")
df_modelo.head()

,FECHA,COD_MUNICIPIO,MUNICIPIO,POBLACION_TOTAL,PROP_ENVEJECIMIENTO,TASA_DEMANDA,TASA_PARO,SERVICIOS_TURISTICOS,SERVICIOS_SANIDAD,ESCUELAS,EMPRESAS,PROP_POBLACION_EXTRANJERA,DISTANCIA_CAPITAL
0,2012,42001,42001 ABEJAR,367,0.318801,13.004484,11.210762,27.0,2.0,0.0,18.0,0.073569,27.4
1,2012,42003,42003 ADRADAS,70,0.471429,10.810811,5.405405,4.0,2.0,0.0,6.0,0.000000,54.3
2,2012,42004,42004 AGREDA,3177,0.246774,12.740510,10.816433,5.0,1.0,3.0,189.0,0.107963,57.1
3,2012,42006,42006 ALCONABA,190,0.268421,7.692308,5.384615,0.0,3.0,0.0,6.0,0.100000,13.0
4,2012,42007,42007 ALCUBILLA DE AVELLANEDA,161,0.540373,8.219178,6.849315,2.0,4.0,0.0,4.0,0.093168,78.9


In [36]:
df_estructura = df_modelo[df_modelo["FECHA"] == 2024].copy()

vars_estructura = [
    "COD_MUNICIPIO",
    "MUNICIPIO",
    "POBLACION_TOTAL",
    "PROP_ENVEJECIMIENTO",
    "EMPRESAS",
    "SERVICIOS_SANIDAD",
    "ESCUELAS",
    "DISTANCIA_CAPITAL"
]

df_estructura = df_estructura[vars_estructura]

Se integran las proyecciones de población a 2030 con las variables estructurales del año base.

In [38]:
df_indice = df_estructura.merge(
    df_2030[
        ["COD_MUNICIPIO", "POBLACION_PROYECTADA"]
    ],
    on="COD_MUNICIPIO",
    how="inner"
)

df_indice = df_indice.rename(
    columns={"POBLACION_PROYECTADA": "POBLACION_2030_PROY"}
)

Se calcula la variación absoluta y relativa de la población proyectada entre 2024 y 2030, como indicador clave de riesgo demográfico futuro.

In [41]:
df_indice["VAR_2024_2030"] = (
    df_indice["POBLACION_2030_PROY"] -
    df_indice["POBLACION_TOTAL"]
)

df_indice["VAR_2024_2030_PCT"] = (
    df_indice["VAR_2024_2030"] /
    df_indice["POBLACION_TOTAL"]
) * 100

Se seleccionan las variables finales del índice de vulnerabilidad a partir del marco teórico y de los resultados de las fases previas.

In [45]:
vars_indice = [
    "VAR_2024_2030_PCT",   # dinámica futura
    "PROP_ENVEJECIMIENTO", # demografía
    "EMPRESAS",            # economía estructural
    "SERVICIOS_SANIDAD",   # servicios básicos
    "ESCUELAS",            # servicios básicos
    "DISTANCIA_CAPITAL"    # accesibilidad
]

In [47]:
#Las variables se normalizan mediante percentiles para garantizar su comparabilidad entre municipios.

for var in vars_indice:
    df_indice[var + "_PCT"] = df_indice[var].rank(pct=True)


invertir = [
    "EMPRESAS",
    "SERVICIOS_SANIDAD",
    "ESCUELAS"
]

#Se invierte el sentido de las variables que actúan como factores de resiliencia territorial.
for var in invertir:
    df_indice[var + "_PCT"] = 1 - df_indice[var + "_PCT"]

df_indice["DIM_DEMOGRAFICA"] = df_indice["PROP_ENVEJECIMIENTO_PCT"]
df_indice["DIM_ECONOMICA"] = df_indice["EMPRESAS_PCT"]
df_indice["DIM_SERVICIOS"] = (
    df_indice["SERVICIOS_SANIDAD_PCT"] +
    df_indice["ESCUELAS_PCT"]
) / 2
df_indice["DIM_ACCESIBILIDAD"] = df_indice["DISTANCIA_CAPITAL_PCT"]
df_indice["DIM_DINAMICA_FUTURA"] = df_indice["VAR_2024_2030_PCT_PCT"]

El índice de vulnerabilidad se calcula como la media de las dimensiones definidas, asignando el mismo peso a cada una para evitar decisiones arbitrarias.

In [50]:
df_indice["INDICE_VULNERABILIDAD"] = (
    df_indice["DIM_DEMOGRAFICA"] +
    df_indice["DIM_ECONOMICA"] +
    df_indice["DIM_SERVICIOS"] +
    df_indice["DIM_ACCESIBILIDAD"] +
    df_indice["DIM_DINAMICA_FUTURA"]
) / 5

In [52]:
df_indice["NIVEL_VULNERABILIDAD"] = pd.qcut(
    df_indice["INDICE_VULNERABILIDAD"],
    q=3,
    labels=["Baja", "Media", "Alta"]
)

In [54]:
df_indice.head()

,COD_MUNICIPIO,MUNICIPIO,POBLACION_TOTAL,PROP_ENVEJECIMIENTO,EMPRESAS,SERVICIOS_SANIDAD,ESCUELAS,DISTANCIA_CAPITAL,POBLACION_2030_PROY,VAR_2024_2030,...,SERVICIOS_SANIDAD_PCT,ESCUELAS_PCT,DISTANCIA_CAPITAL_PCT,DIM_DEMOGRAFICA,DIM_ECONOMICA,DIM_SERVICIOS,DIM_ACCESIBILIDAD,DIM_DINAMICA_FUTURA,INDICE_VULNERABILIDAD,NIVEL_VULNERABILIDAD
0,42001,42001 ABEJAR,386,0.248042,23.0,1.0,0.0,27.4,291,-95,...,0.678571,0.554945,0.178571,0.115385,0.109890,0.616758,0.178571,0.087912,0.221703,Baja
1,42003,42003 ADRADAS,58,0.328125,8.0,2.0,0.0,54.3,74,16,...,0.329670,0.554945,0.615385,0.326923,0.282967,0.442308,0.615385,0.686813,0.470879,Media
2,42004,42004 AGREDA,3094,0.256726,186.0,2.0,3.0,57.1,3179,85,...,0.329670,0.038462,0.686813,0.131868,0.021978,0.184066,0.686813,0.384615,0.281868,Baja
3,42006,42006 ALCONABA,199,0.298507,11.0,4.0,0.0,13.0,407,208,...,0.118132,0.554945,0.027473,0.214286,0.197802,0.336538,0.027473,0.934066,0.342033,Baja
4,42007,42007 ALCUBILLA DE AVELLANEDA,107,0.518868,5.0,4.0,0.0,78.9,160,53,...,0.118132,0.554945,0.936813,0.862637,0.362637,0.336538,0.936813,0.818681,0.663462,Alta


In [57]:
df_indice[
    ["COD_MUNICIPIO", "MUNICIPIO", "NIVEL_VULNERABILIDAD"]
].to_csv(
    "indice_vulnerabilidad.csv",
    index=False
)

In [69]:
df_indice.to_csv(
    "indice_vulnerabilidad_detalle.csv",
    index=False
)